In [5]:
import geopandas as gpd
import pandas as pd
import geopy.geocoders
from geopy.geocoders import Nominatim
import certifi
import ssl

# Specify the directory path
folder_path = "/Users/wendydewit/MDA"

mug_loc = pd.read_parquet(f"{folder_path}/mug_locations.parquet.gzip", engine="pyarrow") 
mug_loc.dropna(inplace=True)

# Create a custom SSL context with CA certificates
context = ssl.create_default_context(cafile=certifi.where())

# Initialize Nominatim geocoder with custom SSL context
geopy.geocoders.options.default_ssl_context = context
geolocator = Nominatim(user_agent="my_geocoder")

# Assuming you have a DataFrame named `mug_loc` with addresses stored in a column named 'address_campus'
geocoded_data = []
addresses_not_geocoded = []  # List to store addresses that were not successfully geocoded

for index, row in mug_loc.iterrows():
    address = row['address_campus']
    location = geolocator.geocode(address)
    if location:
        geocoded_data.append({'address': address,
                              'latitude': location.latitude,
                              'longitude': location.longitude})
    else:
        addresses_not_geocoded.append(address)  # Store addresses that were not successfully geocoded
        print(f"Unable to geocode address: {address}")

# Create a GeoDataFrame from the geocoded data
gdf = gpd.GeoDataFrame(geocoded_data, geometry=gpd.points_from_xy([data['longitude'] for data in geocoded_data],
                                                                   [data['latitude'] for data in geocoded_data]))
print(gdf)

# Print out addresses that were not successfully geocoded
print("\nAddresses not successfully geocoded:")
for address in addresses_not_geocoded:
    print(address)


Unable to geocode address: AV. B. DE CONSTANTINOPLE 5


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=RUE+DE+LA+STATION+25&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [6]:
import folium

# Create a map centered at the mean coordinates of the geocoded data
mean_latitude = gdf['latitude'].mean()
mean_longitude = gdf['longitude'].mean()
map_center = (mean_latitude, mean_longitude)
mymap = folium.Map(location=map_center, zoom_start=10)

# Add markers for each geocoded point
for idx, row in gdf.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['address']).add_to(mymap)

# Display the map
mymap

NameError: name 'gdf' is not defined